# Lightweight Fine-Tuning Project

This project is to classify emotions using a foundation model (GPT2). The purpose is to compare before and after light weight fine-tuning, how the model performances. 




here are the choices for PEFT techique, foundation model used, evaluation approach and the dataset for fine-tuning:

* PEFT technique: Lora techique
* Model: GPT-2
* Evaluation approach: Classification evaluation approaches such as accuracy, confusion matrix, auc, f1 score, precision-recall curve
* Fine-tuning dataset: dair-ai/emotion

In [1]:
# imports modules

from datasets import load_dataset
from collections import Counter

/opt/anaconda3/envs/lightweightFT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load the dataset dair-ai/emotion from datasets 

* Three splits in the dataset including train, validation and test
* We have 6 class labels - what are they?


In [2]:
dataset = load_dataset("dair-ai/emotion")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [3]:
# number of labels
counts = Counter(dataset["train"]["label"])
sorted_counts = sorted(counts.items())
print(sorted_counts)

[(0, 4666), (1, 5362), (2, 1304), (3, 2159), (4, 1937), (5, 572)]


In [4]:
for entry in dataset["train"].select(range(3)):
    text = entry["text"]
    label = entry["label"]
    print(f"label={label}, text={text}")

label=0, text=i didnt feel humiliated
label=0, text=i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake
label=3, text=im grabbing a minute to post i feel greedy wrong


In [5]:
# load GPT-2 and tokenizer and evaluate on the test set

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load GPT-2 tokenizer and model

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=6)


/opt/anaconda3/envs/lightweightFT/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length", truncation=True)

tokenized_dataset = {}
splits = ["train", "validation", "test"]
for split in splits:
    tokenized_dataset[split] = dataset[split].map(tokenize_function, batched=True)

tokenized_dataset["train"]

Map: 100%|██████████| 2000/2000 [00:00<00:00, 4969.76 examples/s]


Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16000
})

In [ ]:
# Prepare dataset for pytorch

from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader



## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.